In [7]:
from docx import Document
from sentence_transformers import SentenceTransformer, util
from difflib import SequenceMatcher
import requests
import torch

# ✅ Device setup for semantic model
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Sentence embedding model (runs on CPU or MPS)
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

import requests

def generate_with_ollama(prompt):
    res = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "llama3.2:3b", "prompt": prompt}
    )
    return res.json()["response"]

# ✅ Ollama generation using DeepSeek
def generate_text(prompt, max_tokens=100):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "llama3.2:3b",  # Or deepseek-coder if that’s the model you pulled
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.7,
                    "top_k": 50,
                    "top_p": 0.9,
                    "num_predict": max_tokens
                }
            }
        )
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"⚠️ Error generating text from Ollama: {e}")
        return ""

# ✅ Resume bullet rewriting logic
def is_similar(a, b, threshold=0.85):
    return SequenceMatcher(None, a, b).ratio() >= threshold

def contains_keywords(text, keywords):
    return all(k.lower() in text.lower() for k in keywords)

def rewrite_bullet(jd, bullet, title, skills):
    prompt = (
        f"You are a professional resume writer.\n"
        f"Job Title: {title}\n"
        f"Skills: {skills}\n"
        f"Job Description: {jd}\n"
        f"Original Bullet: {bullet}\n\n"
        f"Rewrite it in a professional, concise way, under 15 words, starting with an action verb, aligned with the job description. "
        f"Only output the new bullet point."
    )
    try:
        output = generate_text(prompt)
        rewritten = output.strip().split("\n")[-1].strip()
        if not rewritten or is_similar(bullet, rewritten) or len(rewritten) < 5:
            return bullet
        return rewritten
    except Exception as e:
        print(f"⚠️ Error rewriting bullet: {e}")
        return bullet

# ✅ Read and write resume functions
def extract_bullets(docx_path):
    doc = Document(docx_path)
    return [p.text.strip() for p in doc.paragraphs if p.style.name.startswith("List") and len(p.text.strip()) > 20], doc

def write_bullets(doc, rewritten_bullets, output_path="tailored_resume_cleaned.docx"):
    bullet_idx = 0
    for p in doc.paragraphs:
        if p.style.name.startswith("List") and len(p.text.strip()) > 20:
            if bullet_idx < len(rewritten_bullets):
                p.text = rewritten_bullets[bullet_idx]
                bullet_idx += 1
    doc.save(output_path)
    return output_path

# ✅ Input Parameters
resume_path = "Devarshi_Mahajan.docx"
title = "Software Developer"
skills = "GraphQL, Golang, Java, C++"
job_description = (
    "We are hiring a Software Developer proficient in GraphQL, Golang, Java, and C++. "
    "The ideal candidate has experience with scalable backend systems, API development, "
    "and performance optimization. Familiarity with distributed systems and microservices is a plus."
)
keywords = ["graphql", "golang", "java", "c++", "api", "microservices", "scalable", "backend"]


# ✅ Process Resume
bullets, doc = extract_bullets(resume_path)
jd_vec = semantic_model.encode(job_description, convert_to_tensor=True)

rewritten_bullets = []
for bullet in bullets:
    bullet_vec = semantic_model.encode(bullet, convert_to_tensor=True)
    similarity = util.cos_sim(bullet_vec, jd_vec).item()

    if similarity < 0.75 or not contains_keywords(bullet, keywords):
        rewritten = rewrite_bullet(job_description, bullet, title, skills)
    else:
        rewritten = bullet

    rewritten_bullets.append(rewritten)

# ✅ Save Tailored Resume
final_path = write_bullets(doc, rewritten_bullets)
print(f"✅ Tailored resume saved as: {final_path}")


Using device: mps
✅ Tailored resume saved as: tailored_resume_cleaned.docx
